In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.cm as cm
from bokeh.plotting import figure, output_file, show
from bokeh.io import export_png
from bokeh.io.export import get_screenshot_as_png
import geopandas

ModuleNotFoundError: No module named 'geopandas'

In [ ]:
tweets = pd.read_csv('../TP1-Organizacion-de-Datos/csv/train.csv')

In [ ]:
tweets.sample(n = 5)

In [ ]:
ciudades = pd.read_csv('../TP1-Organizacion-de-Datos/csv/worldcities.csv')

In [ ]:
ciudades.sample(n = 5)

In [ ]:
tweets['location'].isnull().sum()

In [ ]:
tweets['location'].isnull().mean()

In [ ]:
location_list = tweets['location'].tolist()

In [ ]:
ciudades_ocurr = tweets['location'].value_counts()

In [ ]:
ciudades_ocurr

In [ ]:
ciudades_list = ciudades['city_ascii']
ciudades_list = pd.DataFrame(ciudades_list)

In [ ]:
ciudades_list['ocurrencia'] = 0

In [ ]:
ciudades_list

In [ ]:
ciudades_list.shape

In [ ]:
ciudades_list['city_ascii'].count()

In [ ]:
ciudades_list['veracidad'] = 0
ciudades_list['cant_verdaderos'] = 0

In [ ]:
ciudades_list['veracidad'] = ciudades_list['veracidad'].astype(float)

In [ ]:
ciudades_list.dtypes

In [ ]:
tienenLaCiudad = pd.DataFrame(columns=('tweetDeLaCiudad', 'V/F'))
tienenLaCiudad['V/F'] = tweets['target']

Comparo cada ciudad del mundo con los tweets y busco si el nombre de la ciudad esta incluido en el campo "Location" de cada Tweet

In [ ]:
#CUIDADO AL EJECUTAR ESTO xD
for x in range(ciudades_list['city_ascii'].count()):
    ciudad = ciudades_list.loc[x, 'city_ascii']
    if (len(ciudad) > 4):
        tienenLaCiudad['tweetDeLaCiudad'] = tweets.loc[:, 'location'].str.contains(ciudad, regex = False)
    else:
        tienenLaCiudad['tweetDeLaCiudad'] = (tweets.loc[:, 'location'] == ciudad)
        
    cant_tweets = tienenLaCiudad[tienenLaCiudad['tweetDeLaCiudad'] == True]['tweetDeLaCiudad'].count()
    verdaderos = tienenLaCiudad[(tienenLaCiudad['V/F'] == 1) & (tienenLaCiudad['tweetDeLaCiudad'] == True)]['V/F'].count()
    
    ciudades_list.loc[x, 'cant_verdaderos'] = verdaderos
    ciudades_list.loc[x, 'ocurrencia'] = cant_tweets
    if (cant_tweets != 0):
        ciudades_list.loc[x, 'veracidad'] = float(verdaderos)/float(cant_tweets)

In [ ]:
ciudades_list

In [ ]:
Ciudades_Mas_10_Tweets = (ciudades_list.loc[:,'ocurrencia'] > 10)

In [ ]:
Top_populares = ciudades_list.loc[Ciudades_Mas_10_Tweets]

In [ ]:
Top_populares = Top_populares.sort_values('ocurrencia', ascending = False)

In [ ]:
Top_populares = Top_populares.drop_duplicates(['city_ascii'])

In [ ]:
Top_populares

In [ ]:
saltos = np.linspace(0.3, 0.7, 48)
colores = (cm.get_cmap('YlOrBr'))(saltos)
Top_populares = Top_populares.sort_values('ocurrencia')
Top_populares_plot = Top_populares.plot(kind='bar', y ='ocurrencia', x = 'city_ascii', figsize=(15, 30), color=colores, width=0.85)

plt.xticks(np.arange(0, 48, 1.0))
plt.tick_params(axis='y', length=0)

#Top_populares_plot.spines['right'].set_visible(False)
#Top_populares_plot.spines['top'].set_visible(False)
#Top_populares_plot.spines['left'].set_visible(False)
#Top_populares_plot.spines['bottom'].set_visible(False)

lineas = Top_populares_plot.get_xticks()
for i in lineas:
      Top_populares_plot.axvline(x=i, linestyle='--', alpha=0.4, color='#eeeeee')
    
plt.gca().invert_xaxis()

Top_populares_plot.set_xlabel("Ciudad", labelpad=20, weight='bold', size=16)
Top_populares_plot.set_ylabel("Cantidad de Tweets", labelpad=20, weight='bold', size=16)
Top_populares_plot.set_title("Ciudades con mayor cantidad de Tweets", weight='bold', size=20)

In [ ]:
saltos = np.linspace(0.3, 0.7, 50)
colores = (cm.get_cmap('YlOrBr'))(saltos)
Top_populares = Top_populares.sort_values('veracidad')
Top_populares_plot = Top_populares.plot(kind='bar', y ='veracidad', x = 'city_ascii', figsize=(15, 30), color=colores, width=0.85)

plt.xticks(np.arange(0, 47, 1), rotation = 80)
plt.tick_params(axis='y', length=0)

#Top_populares_plot.spines['right'].set_visible(False)
#Top_populares_plot.spines['top'].set_visible(False)
#Top_populares_plot.spines['left'].set_visible(False)
#Top_populares_plot.spines['bottom'].set_visible(False)

lineas = Top_populares_plot.get_xticks()
for i in lineas:
      Top_populares_plot.axvline(x=i, linestyle='--', alpha=0.4, color='#eeeeee')

plt.gca().invert_xaxis()
        
Top_populares_plot.set_xlabel("Ciudad", labelpad=20, weight='bold', size=16)
Top_populares_plot.set_ylabel("Veracidad promedio", labelpad=20, weight='bold', size=16)
Top_populares_plot.set_title("Veracidad promedio de los Tweets por ciudad", weight='bold', size=20)

In [ ]:
ciudades_list['Pais'] = ciudades['country']

In [ ]:
ciudades_list

In [ ]:
ciudades_list = ciudades_list.drop_duplicates('city_ascii', keep = 'first')

In [ ]:
ciudades_list.shape

In [ ]:
Tweets_Paises = pd.DataFrame({'Cantidad_Tweets': 0, 'Pais': ciudades['country'], 'Veracidad': 0, 'Cantidad_Tweets_Verdaderos': 0})

In [ ]:
Tweets_Paises['Veracidad'] = Tweets_Paises['Veracidad'].astype(float)

In [ ]:
Tweets_Paises['Pais'].describe()

In [ ]:
Tweets_Paises = Tweets_Paises.drop_duplicates(['Pais'])
Tweets_Paises = Tweets_Paises.reset_index(drop = True)

In [ ]:
Tweets_Paises.dtypes

In [ ]:
ciudades_list.dtypes

In [ ]:
ciudades_list

In [ ]:
#CUIDADO AL EJECUTAR ESTO xD
for w in range(Tweets_Paises['Pais'].count()):
    
    pais = Tweets_Paises.loc[w, 'Pais']
    
    la_ciudad_es_del_pais = ciudades_list.loc[:, 'Pais'].str.contains(pais , regex = False)
    
    ciudades_del_pais = ciudades_list.loc[la_ciudad_es_del_pais]
    
    total_tweets_pais = ciudades_del_pais['ocurrencia'].sum()

    total_tweets_verdaderos_pais = ciudades_del_pais['cant_verdaderos'].sum()
    
    Tweets_Paises.loc[w, 'Cantidad_Tweets'] = total_tweets_pais
    
    Tweets_Paises.loc[w, 'Cantidad_Tweets_Verdaderos'] = total_tweets_verdaderos_pais
    
    if (total_tweets_pais != 0):
        
        Tweets_Paises.loc[w, 'Veracidad'] = float(total_tweets_verdaderos_pais)/float(total_tweets_pais)

In [ ]:
Tweets_Paises

In [ ]:
Tweets_Paises['Cantidad_Tweets'].sum()

In [ ]:
paises_mas_10_tweets = (Tweets_Paises.loc[:,'Cantidad_Tweets'] > 10)

In [ ]:
paises_mas_10_tweets = Tweets_Paises.loc[paises_mas_10_tweets]

In [ ]:
paises_mas_10_tweets = paises_mas_10_tweets.sort_values('Cantidad_Tweets')

In [ ]:
saltos = np.linspace(0.3, 0.7, 20)
colores = (cm.get_cmap('YlOrBr'))(saltos)

paises_mas_10_tweets_plot = paises_mas_10_tweets.plot(kind='bar', y ='Cantidad_Tweets', x = 'Pais', figsize=(15, 15), color=colores, width=0.85)

plt.xticks(np.arange(0, 19, 1.0))
plt.tick_params(axis='y', length=0)

#paises_mas_10_tweets_plot.spines['right'].set_visible(False)
#paises_mas_10_tweets_plot.spines['top'].set_visible(False)
#paises_mas_10_tweets_plot.spines['left'].set_visible(False)
#paises_mas_10_tweets_plot.spines['bottom'].set_visible(False)

lineas = paises_mas_10_tweets_plot.get_xticks()
for i in lineas:
      paises_mas_10_tweets_plot.axvline(x=i, linestyle='--', alpha=0.4, color='#eeeeee')
        
plt.xticks(fontsize = 10, rotation = 40)
plt.yticks(fontsize = 10)

plt.gca().invert_xaxis()

paises_mas_10_tweets_plot.set_xlabel("Pais", labelpad=20, weight='bold', size=16)
paises_mas_10_tweets_plot.set_ylabel("Cantidad de Tweets", labelpad=20, weight='bold', size=16)
paises_mas_10_tweets_plot.set_title("Paises con mayor cantidad de Tweets", weight='bold', size=20)

In [ ]:
saltos = np.linspace(0.3, 0.7, 20)
colores = (cm.get_cmap('YlOrBr'))(saltos)

paises_mas_10_tweets_plot = paises_mas_10_tweets.plot(kind='bar', y ='Veracidad', x = 'Pais', figsize=(30,10), color=colores, width=0.80)

paises_mas_10_tweets_plot.spines['right'].set_visible(False)
paises_mas_10_tweets_plot.spines['top'].set_visible(False)
paises_mas_10_tweets_plot.spines['left'].set_visible(False)
paises_mas_10_tweets_plot.spines['bottom'].set_visible(False)

lineas = paises_mas_10_tweets_plot.get_xticks()
for i in lineas:
      paises_mas_10_tweets_plot.axvline(x=i, linestyle='--', alpha=0.4, color='#eeeeee')
             
plt.xticks(fontsize = 15, rotation = 40)
plt.yticks(fontsize = 15)

paises_mas_10_tweets_plot.set_xlabel("Pais", labelpad=20, weight='bold', size=16)
paises_mas_10_tweets_plot.set_ylabel("Veracidad promedio", labelpad=20, weight='bold', size=16)
paises_mas_10_tweets_plot.set_title("Veracidad promedio de los Tweets por pais", weight='bold', size=20)

Veracidad de los tweets cuando la locacion es "valida"

In [ ]:
Tweets_Paises['Cantidad_Tweets'].sum()

In [ ]:
Tweets_Paises['Cantidad_Tweets_Verdaderos'].sum()

In [ ]:
float(Tweets_Paises['Cantidad_Tweets_Verdaderos'].sum())/float(Tweets_Paises['Cantidad_Tweets'].sum())

------------------------------------------------------------------------------------------------------------------

In [ ]:
Tweets_Verdaderos_Locations_Invalidas = (tweets['target'].sum() - Tweets_Paises['Cantidad_Tweets_Verdaderos'].sum())

In [ ]:
Tweets_Verdaderos_Locations_Invalidas

In [ ]:
Tweets_Totales_Locations_Invalidas = (tweets['id'].count() - Tweets_Paises['Cantidad_Tweets'].sum())

In [ ]:
Tweets_Totales_Locations_Invalidas

In [ ]:
float(Tweets_Verdaderos_Locations_Invalidas)/float(Tweets_Totales_Locations_Invalidas)

Cuando la location es invalida hay una ligera diferencia entre la veracidad de los Tweets

In [ ]:
ciudades_list = ciudades_list.reset_index(drop = True)

In [ ]:
ciudades = ciudades.drop_duplicates('city_ascii', keep = 'first')

In [ ]:
ciudades = ciudades.reset_index(drop = True) # A ciudades_list ya le habia hecho el drop_duplicates antes.

In [ ]:
ciudades_list['cant_habitantes'] = ciudades['population']

In [ ]:
ciudades_list

In [ ]:
Ciudades_Mas_De_10_Tweets = (ciudades_list.loc[:,'ocurrencia'] > 5)

In [ ]:
fig, ax= plt.subplots(figsize = (10, 10))
Plot = sns.scatterplot(x = "veracidad", y = "cant_habitantes", data = ciudades_list.loc[Ciudades_Mas_De_10_Tweets], ax = ax)

Plot.set_xlabel("Veracidad", labelpad=20, weight='bold', size=16)
Plot.set_ylabel("Cantidad de habitantes", labelpad=20, weight='bold', size=16)
Plot.set_title("Cantidad de habitantes Vs. Veracidad", weight='bold', size=20)


In [ ]:
fig, ax= plt.subplots(figsize = (10, 10))
Plot = sns.scatterplot(x = "veracidad", y = "ocurrencia", data = ciudades_list.loc[Ciudades_Mas_De_10_Tweets], ax = ax)

Plot.set_xlabel("Veracidad", labelpad=20, weight='bold', size=16)
Plot.set_ylabel("Cantidad de Tweets", labelpad=20, weight='bold', size=16)
Plot.set_title("Cantidad de Tweets Vs. Veracidad (Por ciudad)", weight='bold', size=20)

In [ ]:
cmap = cm.get_cmap('YlOrBr')
paleta = cmap(np.linspace(0.3, 0.9,100))

colores_rgb = []
for i in paleta:
    #convierto rgba en rgb
    aux = list(i)
    del aux[3]
    aux = (int(x * 255) for x in aux)
    aux =tuple(aux)
    colores_rgb.append((aux))

In [ ]:
type(colores_rgb)

In [ ]:
# prepare some data
x = ciudades_list.loc[Ciudades_Mas_De_10_Tweets]['veracidad']
y = ciudades_list.loc[Ciudades_Mas_De_10_Tweets]['ocurrencia']
radii = np.sqrt(ciudades_list.loc[Ciudades_Mas_De_10_Tweets]['cant_habitantes'])/7e4
colors = [
    "#%02x%02x%02x" % i for i in colores_rgb
]

# output to static HTML file (with CDN resources)
output_file("Veracidad_Vs_Cantidad de Tweets.html", title="color_scatter.py example", mode="cdn")

TOOLS = "crosshair,pan,wheel_zoom,box_zoom,reset,box_select,lasso_select"

# create a new plot with the tools above, and explicit ranges
p = figure(tools=TOOLS, x_range=(0, 1), y_range=(0, 150), width=1000, height=1000)

# add a circle renderer with vectorized colors and sizes
p.circle(x, y, radius=radii, fill_color=colors, fill_alpha=0.6, line_color=None)

p.title.text = "Veracidad Vs. Cantidad de Tweets (Por ciudad)"
p.legend.location = "top"
p.xaxis.axis_label = 'Veracidad promedio'
p.yaxis.axis_label = 'Cantidad de Tweets'

# show the results
show(p)

In [ ]:
# prepare some data
x = ciudades_list.loc[Ciudades_Mas_De_10_Tweets]['veracidad']
y = ciudades_list.loc[Ciudades_Mas_De_10_Tweets]['cant_habitantes']/1e5
radii = np.sqrt(ciudades_list.loc[Ciudades_Mas_De_10_Tweets]['ocurrencia'])/1e2
# output to static HTML file (with CDN resources)
output_file("Veracidad_Vs_Cantidad de Habitantes.html", title="color_scatter.py example", mode="cdn")

TOOLS = "crosshair,pan,wheel_zoom,box_zoom,reset,box_select,lasso_select"

# create a new plot with the tools above, and explicit ranges
q = figure(tools=TOOLS, x_range=(0, 1), y_range=(0, 150), width=1000, height=1000)

# add a circle renderer with vectorized colors and sizes
q.circle(x, y, radius=radii, fill_color=colors, fill_alpha=0.6, line_color=None)

q.title.text = "Veracidad Vs. Cantidad de Habitantes (Por ciudad)"
q.legend.location = "top"
q.xaxis.axis_label = 'Veracidad promedio'
q.yaxis.axis_label = 'Cantidad de Habitantes (x1e5)'

# show the results
show(q)

In [ ]:
world = geopandas.read_file(geopandas.datasets.get_path('naturalearth_lowres'))

In [ ]:
world.head()

In [ ]:
world.count()

In [ ]:
iso3 = pd.DataFrame({'iso3': ciudades['iso3']})
iso3 = iso3.drop_duplicates('iso3', keep = 'first')
iso3 = iso3.reset_index(drop = True)
Tweets_Paises['iso3']=iso3['iso3']

In [ ]:
#CUIDADO AL EJECUTAR ESTO xD
for w in range(world['name'].count()):
    
    code_pais = world.loc[w, 'iso_a3']
    
    el_code_es_del_pais = Tweets_Paises.loc[:, 'iso3'] == code_pais
    
    codigo_del_pais = Tweets_Paises.loc[el_code_es_del_pais]
    
    world.loc[w, 'veracidad'] = np.nan
    
    if codigo_del_pais['Cantidad_Tweets'].sum() > 5:
        
        world.loc[w, 'veracidad'] = codigo_del_pais['Veracidad'].sum()

In [ ]:
world

In [ ]:
fig, ax = plt.subplots(figsize = (150, 20))
world = world[world.name!="Antarctica"]

ax.patch.set_facecolor('#eeeeee')

world.plot(column='veracidad', ax = ax, legend = True, cmap='OrRd')